In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.path import Path
import pandas as pd
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Set seeds for reproducibility
torch.manual_seed(0)
np.random.seed(0)

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define the NN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(2, 64),
            nn.Tanh(),
            nn.Linear(64, 128),
            nn.Tanh(),
            nn.Linear(128, 256),
            nn.Tanh(),
            nn.Linear(256, 128),
            nn.Tanh(),
            nn.Linear(128, 64),
            nn.Tanh(),
            nn.Linear(64, 2)  # Output 2 values: ux and uy
        )

    def forward(self, x):
        return self.layers(x)

# Define the problem domain using vertices
vertices = np.array([
    [0, 0], [0, 3], [3, 3], [5, 2], [7, 2], [7, 0], [0, 0]
], dtype=np.float32)
path = Path(vertices)

def in_domain(x, y):
    points = np.column_stack((x.cpu().numpy(), y.cpu().numpy()))
    return torch.tensor(path.contains_points(points), dtype=torch.bool, device=device)

# Define boundary conditions
def BC_bottom(x, y):
    return ((y <= 0.1) & (x >= 0) & (x <= 7)).squeeze()

def BC_left(x, y):
    return ((x <= 0.1) & (y >= 0) & (y <= 3)).squeeze()

def BC_top(x, y):
    return (((y >= 2.9) & (x >= 0) & (x <= 3)) | 
            ((y - 4.5) * -2 == (x - 0)) & (x > 3) & (x < 5) |
            (y == 2) & (x >= 5) & (x <= 7)).squeeze()

def BC_right(x, y):
    return ((x >= 6.9) & (y >= 0) & (y <= 2)).squeeze()


def BC(xy, net):    
    x, y = xy[:, 0], xy[:, 1]
    
    uv = net(xy)
    u, v = uv[:, 0], uv[:, 1]
    
    bc_b = BC_bottom(x, y)     
    bc_l = BC_left(x, y)
    bc_t = BC_top(x, y)
    bc_r = BC_right(x, y)
    
    E = 5  # <---------------Young's modulus
    nu = 0.3  # <---------------Poisson's ratio
    
    loss = torch.mean(u[bc_b]**2 + v[bc_b]**2)  # ux = uy = 0 on bottom
    
    # Stress-free condition at the left (ux = 0, σxy = 0)
    if torch.any(bc_l):
        xy_left = xy[bc_l]
        xy_left.requires_grad_(True)
        uv_left = net(xy_left)
        u_left, v_left = uv_left[:, 0], uv_left[:, 1]
        
        u_y_left = torch.autograd.grad(u_left.sum(), xy_left, create_graph=True)[0][:, 1]
        v_x_left = torch.autograd.grad(v_left.sum(), xy_left, create_graph=True)[0][:, 0]
        
        sigma_xy_left = E / (2 * (1 + nu)) * (u_y_left + v_x_left)
        loss += torch.mean(u_left**2 + sigma_xy_left**2)
    
    # Stress-free condition at the right (ux = 0, σxy = 0)
    if torch.any(bc_r):
        xy_right = xy[bc_r]
        xy_right.requires_grad_(True)
        uv_right = net(xy_right)
        u_right, v_right = uv_right[:, 0], uv_right[:, 1]
        
        u_y_right = torch.autograd.grad(u_right.sum(), xy_right, create_graph=True)[0][:, 1]
        v_x_right = torch.autograd.grad(v_right.sum(), xy_right, create_graph=True)[0][:, 0]
        
        sigma_xy_right = E / (2 * (1 + nu)) * (u_y_right + v_x_right)
        loss += torch.mean(u_right**2 + sigma_xy_right**2)
    
    # Stress-free condition at the top (σxx = 0, σyy = 0, σxy = 0)
    if torch.any(bc_t):
        xy_top = xy[bc_t]
        xy_top.requires_grad_(True)
        uv_top = net(xy_top)
        u_top, v_top = uv_top[:, 0], uv_top[:, 1]
        
        u_x_top = torch.autograd.grad(u_top.sum(), xy_top, create_graph=True)[0][:, 0]
        u_y_top = torch.autograd.grad(u_top.sum(), xy_top, create_graph=True)[0][:, 1]
        v_x_top = torch.autograd.grad(v_top.sum(), xy_top, create_graph=True)[0][:, 0]
        v_y_top = torch.autograd.grad(v_top.sum(), xy_top, create_graph=True)[0][:, 1]
        
        sigma_xx_top = E / (1 - nu**2) * (u_x_top + nu * v_y_top)
        sigma_yy_top = E / (1 - nu**2) * (v_y_top + nu * u_x_top)
        sigma_xy_top = E / (2 * (1 + nu)) * (u_y_top + v_x_top)
        loss += torch.mean(sigma_xx_top**2 + sigma_yy_top**2 + sigma_xy_top**2)

    return loss


def load_fem_data(filename='FEM4_data.csv'): #ใช้ X,Y เป็น Reference Coordinate (Optional)
    df = pd.read_csv(filename)
    x = torch.tensor(df['X'].values, dtype=torch.float32).unsqueeze(1).to(device)
    y = torch.tensor(df['Y'].values, dtype=torch.float32).unsqueeze(1).to(device)
    return x, y

def generate_training_data(fem_data_file, n_boundary):
    x, y = load_fem_data(fem_data_file)
    
    # points inside the domain
    mask = in_domain(x, y)
    x, y = x[mask].to(device), y[mask].to(device)
    
    # Generate boundary points
    t = torch.linspace(0, 1, n_boundary, device=device).unsqueeze(1)
    
    # Define the boundary segments (สร้างจุด)
    segments = [
        ([0, 0, 0], [0, 3, 3]),  # Left boundary
        ([0, 3, 5, 7, 7], [3, 3, 2, 2, 0]),  # Top and right boundary
        ([7, 0], [0, 0])  # Bottom boundary
    ]
    
    x_b = []
    y_b = []
    
    for segment in segments:
        x_seg = torch.tensor(np.interp(t.cpu().numpy(), np.linspace(0, 1, len(segment[0])), segment[0]), dtype=torch.float32, device=device)
        y_seg = torch.tensor(np.interp(t.cpu().numpy(), np.linspace(0, 1, len(segment[1])), segment[1]), dtype=torch.float32, device=device)
        x_b.append(x_seg)
        y_b.append(y_seg)
    
    x_b = torch.cat(x_b)
    y_b = torch.cat(y_b)
    
    return x.to(device), y.to(device), x_b.to(device), y_b.to(device)

#Define Tension cut off function
def tension_cutoff_yield_functions(sigma_1, sigma_2, sigma_3, sigma_t):
    f4 = sigma_1 - sigma_t
    f5 = sigma_2 - sigma_t
    f6 = sigma_3 - sigma_t
    return torch.stack([f4, f5, f6], dim=-1)
#Define Mohr coulomb yield function
def mohr_coulomb_yield_functions(sigma_1, sigma_2, sigma_3, c, phi, sigma_t):
    sin_phi = torch.sin(phi)
    cos_phi = torch.cos(phi)

    f1a = 0.5 * (sigma_2 - sigma_3) + 0.5 * (sigma_2 + sigma_3) * sin_phi - c * cos_phi
    f1b = 0.5 * (sigma_3 - sigma_2) + 0.5 * (sigma_3 + sigma_2) * sin_phi - c * cos_phi
    f2a = 0.5 * (sigma_3 - sigma_1) + 0.5 * (sigma_3 + sigma_1) * sin_phi - c * cos_phi
    f2b = 0.5 * (sigma_1 - sigma_3) + 0.5 * (sigma_1 + sigma_3) * sin_phi - c * cos_phi
    f3a = 0.5 * (sigma_1 - sigma_2) + 0.5 * (sigma_1 + sigma_2) * sin_phi - c * cos_phi
    f3b = 0.5 * (sigma_2 - sigma_1) + 0.5 * (sigma_2 + sigma_1) * sin_phi - c * cos_phi

    f_mc = torch.stack([f1a, f1b, f2a, f2b, f3a, f3b], dim=-1)
    
    f_tension = tension_cutoff_yield_functions(sigma_1, sigma_2, sigma_3, sigma_t)
    
    yield_values = torch.cat([f_mc, f_tension], dim=-1)
    return yield_values, torch.max(yield_values, dim=-1)[0]
#Define plastic potential
def plastic_potential(sigma_1, sigma_2, sigma_3, psi):
    sin_psi = torch.sin(psi)
    g1a = 0.5 * (sigma_2 - sigma_3) + 0.5 * (sigma_2 + sigma_3) * sin_psi
    g1b = 0.5 * (sigma_3 - sigma_2) + 0.5 * (sigma_3 + sigma_2) * sin_psi
    g2a = 0.5 * (sigma_3 - sigma_1) + 0.5 * (sigma_3 + sigma_1) * sin_psi
    g2b = 0.5 * (sigma_1 - sigma_3) + 0.5 * (sigma_1 + sigma_3) * sin_psi
    g3a = 0.5 * (sigma_1 - sigma_2) + 0.5 * (sigma_1 + sigma_2) * sin_psi
    g3b = 0.5 * (sigma_2 - sigma_1) + 0.5 * (sigma_2 + sigma_1) * sin_psi
    return torch.stack([g1a, g1b, g2a, g2b, g3a, g3b], dim=-1)
    
#Define Elastic matrix
def calculate_D_e(E, nu, device):
    D_e = E / ((1 + nu) * (1 - 2*nu)) * torch.tensor([
        [1-nu, nu, 0],
        [nu, 1-nu, 0],
        [0, 0, (1-2*nu)/2]
    ], device=device)
    return D_e

def calculate_D_ep(D, r, df_dsigma, df_dxi, h):
    D_r = torch.einsum('bij,bj->bi', D, r)
    numerator = torch.einsum('bi,bj->bij', D_r, torch.einsum('bij,bj->bi', D, df_dsigma))  
    
    denominator = torch.einsum('bi,bij,bj->b', df_dsigma, D, r) - torch.einsum('bi,bi->b', df_dxi, h)
    denominator = torch.clamp(denominator, min=1e-8) #Prevent division by zero
    
    D_ep = D - numerator / denominator.unsqueeze(-1).unsqueeze(-1)
    
    return D_ep

#Define principal stress calculation
def calculate_principal_stresses(sigma_xx, sigma_yy, sigma_xy):
    # คำนวณค่าเค้นหลัก
    sigma_avg = (sigma_xx + sigma_yy) / 2
    R = torch.sqrt(((sigma_xx - sigma_yy) / 2) ** 2 + sigma_xy ** 2)

    sigma_1 = sigma_avg + R  # ค่าเค้นหลักที่มากที่สุด (Maximum principal stress)
    sigma_3 = sigma_avg - R  # ค่าเค้นหลักที่น้อยที่สุด (Minimum principal stress)
    return sigma_1, sigma_3

def PDE(x, y, net):
    device = x.device
    xy = torch.cat([x, y], dim=1)
    xy.requires_grad = True
    
    uv = net(xy)
    u, v = uv[:, 0].unsqueeze(1), uv[:, 1].unsqueeze(1)
    
    u_x = torch.autograd.grad(u.sum(), xy, create_graph=True)[0][:, 0].unsqueeze(1)
    u_y = torch.autograd.grad(u.sum(), xy, create_graph=True)[0][:, 1].unsqueeze(1)
    v_x = torch.autograd.grad(v.sum(), xy, create_graph=True)[0][:, 0].unsqueeze(1)
    v_y = torch.autograd.grad(v.sum(), xy, create_graph=True)[0][:, 1].unsqueeze(1)
    
    # Material properties
    E = torch.tensor(5.0, device=device)  # <--------------- Young's modulus 
    nu = torch.tensor(0.3, device=device)  # <---------------Poisson's ratio
    gamma = torch.tensor(1.8, device=device)  # <---------------Unit weight
    c = torch.tensor(3, device=device)  # <---------------Cohesion
    phi = torch.tensor(13, device=device)  # <---------------Friction angle in degrees
    psi = torch.tensor(0.0, device=device)  # <---------------Dilation angle in degrees
    sigma_t = torch.tensor(0.0, device=device)  # <---------------Tension cut-off

    D_e = calculate_D_e(E, nu, device)
    D_e = D_e.unsqueeze(0).expand(x.size(0), -1, -1)

    strain = torch.stack([u_x, v_y, u_y + v_x], dim=-1).squeeze(-2)
    sigma_e = torch.einsum('bij,bj->bi', D_e, strain)
    sigma_xx, sigma_yy, sigma_xy = sigma_e.unbind(dim=-1)

    sigma_1, sigma_3 = calculate_principal_stresses(sigma_xx, sigma_yy, sigma_xy)
    sigma_2 = torch.zeros_like(sigma_1)
    
    yield_values, max_yield = mohr_coulomb_yield_functions(sigma_1, sigma_2, sigma_3, c, phi, sigma_t)
    potential_values = plastic_potential(sigma_1, sigma_2, sigma_3, psi)
    
    df_dsigma = torch.autograd.grad(yield_values.sum(), sigma_e, create_graph=True)[0]
    dg_dsigma = torch.autograd.grad(potential_values.sum(), sigma_e, create_graph=True)[0]
    
    
    df_dxi = torch.zeros_like(df_dsigma[:, :1])
    h = torch.zeros_like(df_dxi)
    
    D_ep = calculate_D_ep(D_e, dg_dsigma, df_dsigma, df_dxi, h)
    
    # Plastic strain increment
    lambda_p = torch.where(max_yield > 0, 
                        1 / (torch.einsum('bi,bij,bj->b', df_dsigma, D_e, dg_dsigma) + 1e-8),
                        torch.zeros_like(max_yield))
    d_epsilon_p = lambda_p.unsqueeze(-1) * dg_dsigma
    
    #Total strain
    epsilon_total = strain + d_epsilon_p
    
    #Stress correction using D_ep
    sigma_corrected = torch.einsum('bij,bj->bi', D_ep, epsilon_total)

    sigma_xx_corrected, sigma_yy_corrected, sigma_xy_corrected = sigma_corrected.unbind(dim=-1)

    f_x = torch.zeros_like(x)
    f_y = -gamma * torch.ones_like(y)

    R_x = torch.autograd.grad(sigma_xx_corrected.sum(), xy, create_graph=True)[0][:, 0].unsqueeze(1) + \
          torch.autograd.grad(sigma_xy_corrected.sum(), xy, create_graph=True)[0][:, 1].unsqueeze(1) + f_x
    R_y = torch.autograd.grad(sigma_xy_corrected.sum(), xy, create_graph=True)[0][:, 0].unsqueeze(1) + \
          torch.autograd.grad(sigma_yy_corrected.sum(), xy, create_graph=True)[0][:, 1].unsqueeze(1) + f_y

    yield_loss = torch.mean(torch.relu(max_yield))
    potential_loss = torch.mean(torch.abs(potential_values))
    pde_loss_x = torch.mean(R_x ** 2)
    pde_loss_y = torch.mean(R_y ** 2)

    return pde_loss_x, pde_loss_y, yield_loss, potential_loss, sigma_corrected, epsilon_total

def phi_c_reduction(net, fem_data_file, initial_sf=1.0, sf_step=0.01, max_iterations=1000):
    device = next(net.parameters()).device
    
    c_initial = torch.tensor(3.0, device=device)
    phi_initial = torch.deg2rad(torch.tensor(13.0, device=device))
    
    current_sf = initial_sf
    
    x, y, x_b, y_b = generate_training_data(fem_data_file, 30000)
    xy = torch.cat([x, y], dim=1).requires_grad_(True)
    
    sf_values = []
    failure_ratios = []
    
    for iteration in range(max_iterations):
        c_reduced = c_initial / current_sf
        phi_reduced = torch.atan(torch.tan(phi_initial) / current_sf)
        
        with torch.enable_grad():
            uv = net(xy)
            u, v = uv[:, 0], uv[:, 1]
            
            u_x = torch.autograd.grad(u.sum(), xy, create_graph=True)[0][:, 0]
            u_y = torch.autograd.grad(u.sum(), xy, create_graph=True)[0][:, 1]
            v_x = torch.autograd.grad(v.sum(), xy, create_graph=True)[0][:, 0]
            v_y = torch.autograd.grad(v.sum(), xy, create_graph=True)[0][:, 1]
            
            E = torch.tensor(5.0, device=device) #<--------------- Young's modulus
            nu = torch.tensor(0.3, device=device) #<--------------- Poisson's ratio
            
            D_e = calculate_D_e(E, nu, device)
            D_e = D_e.unsqueeze(0).expand(x.size(0), -1, -1)
            
            strain = torch.stack([u_x, v_y, u_y + v_x], dim=-1)
            sigma_e = torch.einsum('bij,bj->bi', D_e, strain)
            sigma_xx, sigma_yy, sigma_xy = sigma_e.unbind(dim=-1)
            
            sigma_1, sigma_3 = calculate_principal_stresses(sigma_xx, sigma_yy, sigma_xy)
        
        # Using Mohr-Coulomb criterion with reduced strength parameters
        failure_criterion = (sigma_1 - sigma_3) - (sigma_1 + sigma_3) * torch.sin(phi_reduced) - 2 * c_reduced * torch.cos(phi_reduced)
        
        failure_ratio = torch.sum(failure_criterion > 0) / failure_criterion.numel()
        sf_values.append(current_sf)
        failure_ratios.append(failure_ratio.item())
        
        if failure_ratio > 0.01:  # If more than 1% of points fail
            return current_sf, sf_values, failure_ratios
        
        current_sf += sf_step
    
    return current_sf, sf_values, failure_ratios

def train(net, optimizer, n_epochs, fem_data_file):
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=500, verbose=True)
    
    best_loss = float('inf')
    
    for epoch in range(n_epochs):
        optimizer.zero_grad()
        
        x, y, x_b, y_b = generate_training_data(fem_data_file, 30000)  # 30000 data points
        xy = torch.cat([x, y], dim=1)
        xy_b = torch.cat([x_b, y_b], dim=1)
        
        pde_results = PDE(x, y, net)
        loss_pde_x, loss_pde_y, yield_loss, potential_loss = pde_results[:4]
        loss_bc = BC(xy_b, net)
        
        loss = loss_pde_x + 2 * loss_pde_y + loss_bc + 0.1 * yield_loss + 0.1 * potential_loss
        
        if torch.isnan(loss):
            print(f"NaN loss detected at epoch {epoch+1}")
            print(f"loss_pde_x: {loss_pde_x.item()}, loss_pde_y: {loss_pde_y.item()}")
            print(f"loss_bc: {loss_bc.item()}, yield_loss: {yield_loss.item()}")
            print(f"potential_loss: {potential_loss.item()}")
            break
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(net.parameters(), max_norm=1.0)
        optimizer.step()
        
        scheduler.step(loss)
        
        if loss < best_loss:
            best_loss = loss
            torch.save({
                'net_state_dict': net.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss,
            }, 'best_model.pth')
        
        if (epoch + 1) % 10 == 0:
            print(f'Epoch {epoch+1}/{n_epochs}, Total Loss: {loss.item():.6f}, '
                  f'PDE Loss X: {loss_pde_x.item():.6f}, PDE Loss Y: {loss_pde_y.item():.6f}, '
                  f'BC Loss: {loss_bc.item():.6f}, LR: {optimizer.param_groups[0]["lr"]:.6f}')
            
def calculate_elastic_plastic_status(sigma_1, sigma_3, c, phi, sigma_yy, sigma_xx,sigma_xy):
    sin_phi = torch.sin(torch.deg2rad(phi))
    cos_phi = torch.cos(torch.deg2rad(phi))
    c = 3  # <--------------Cohesion
    phi = torch.tensor(13, device=sigma_1.device, dtype=sigma_1.dtype) #<--------------friction angle
    sigma_avg = (sigma_xx + sigma_yy) / 2
    R = torch.sqrt(((sigma_xx - sigma_yy) / 2) ** 2 + sigma_xy ** 2)

    sigma_1 = (sigma_avg + R)*10  # ค่าเค้นหลักที่มากที่สุด (Maximum principal stress)
    sigma_3 = (sigma_avg - R)*10  # ค่าเค้นหลักที่น้อยที่สุด (Minimum principal stress)

    # Calculate τmob (mobilized shear strength)
    tau_mob = (sigma_1 - sigma_3) / 2
    
    # Calculate τmax (maximum shear strength)
    tau_max = c * cos_phi - ((sigma_1 + sigma_3) / 2) * sin_phi
    
    # Calculate τrel (relative shear strength)
    tau_rel = torch.abs(tau_mob / tau_max)
    
    # Define tolerance for considering status
    tolerance = 0.99
    
    # 0: Elastic, 1: Plastic (Failure)
    status = torch.where(tau_rel > tolerance, torch.ones_like(tau_rel), torch.zeros_like(tau_rel))
    
    return status, tau_rel, tau_max, tau_mob


def plot_results(net, fem_data_file='FEM4_data.csv', output_filename='PiNN4_data.csv'):
    df_fem = pd.read_csv(fem_data_file)
    X = df_fem['X'].values
    Y = df_fem['Y'].values
    
    XY = torch.tensor(np.column_stack([X, Y]), dtype=torch.float32, requires_grad=True).to(device)
    
    with torch.enable_grad():
        UV = net(XY)
        U, V = UV[:, 0], UV[:, 1]
        
        U_x = torch.autograd.grad(U.sum(), XY, create_graph=True)[0][:, 0]
        U_y = torch.autograd.grad(U.sum(), XY, create_graph=True)[0][:, 1]
        V_x = torch.autograd.grad(V.sum(), XY, create_graph=True)[0][:, 0]
        V_y = torch.autograd.grad(V.sum(), XY, create_graph=True)[0][:, 1]
        
        E = 5  # <--------------Young's modulus
        nu = 0.3  # <--------------Poisson's ratio
        c = 3  # <--------------Cohesion
        phi = torch.deg2rad(torch.tensor(13.0, device=device))  # <--------------Friction angle
        
        # Calculate strain components
        epsilon_xx = U_x
        epsilon_yy = V_y
        epsilon_xy = 0.5 * (U_y + V_x)

        # Calculate principal strains
        epsilon_avg = 0.5 * (epsilon_xx + epsilon_yy)
        epsilon_diff = 0.5 * torch.sqrt((epsilon_xx - epsilon_yy)**2 + 4 * epsilon_xy**2)
        epsilon_1 = epsilon_avg + epsilon_diff
        epsilon_3 = epsilon_avg - epsilon_diff

        # Calculate stress components
        sigma_xx = E / (1 - nu**2) * (U_x + nu * V_y)
        sigma_yy = E / (1 - nu**2) * (V_y + nu * U_x)
        sigma_xy = E / (2 * (1 + nu)) * (U_y + V_x)
        
        # Calculate principal stresses
        sigma_avg = (sigma_xx + sigma_yy) / 2
        R = torch.sqrt(((sigma_xx - sigma_yy) / 2) ** 2 + sigma_xy ** 2)

        sigma_1 = sigma_avg + R
        sigma_3 = sigma_avg - R

    # Detach tensors and move to CPU for numpy operations
    with torch.no_grad():
        sigma_1_full = sigma_1.cpu().numpy().squeeze() * 10
        sigma_3_full = sigma_3.cpu().numpy().squeeze() * 10

        # Initialize arrays for sorted stresses
        sigma_1_sorted = np.zeros_like(sigma_1_full)
        sigma_3_sorted = np.zeros_like(sigma_3_full)

        # Sort stresses for each coordinate
        for i in range(len(sigma_1_full)):
            # Sort by absolute value
            sorted_indices = np.argsort(np.abs([sigma_1_full[i], sigma_3_full[i]]))[::-1]
            sigma_1_sorted[i] = -abs([sigma_1_full[i], sigma_3_full[i]][sorted_indices[0]])
            sigma_3_sorted[i] = -abs([sigma_1_full[i], sigma_3_full[i]][sorted_indices[1]])
        
        c = 3  # <--------------Cohesion
        phi = torch.tensor(13, device=sigma_1.device, dtype=sigma_1.dtype) #<-------------- Friction angle

        status, tau_rel, tau_max, tau_mob = calculate_elastic_plastic_status(sigma_1, sigma_3, c, phi, sigma_yy, sigma_xx, sigma_xy)

        status_full = status.cpu().numpy().squeeze()
        tau_rel_full = tau_rel.cpu().numpy().squeeze()
        tau_max_full = tau_max.cpu().numpy().squeeze()
        tau_mob_full = tau_mob.cpu().numpy().squeeze()
        U_full = U.cpu().numpy().squeeze() / 1000     
        V_full = V.cpu().numpy().squeeze() / 1000
        sigma_xx_full = sigma_xx.cpu().numpy().squeeze() * 10
        sigma_yy_full = sigma_yy.cpu().numpy().squeeze() * 10
        sigma_xy_full = sigma_xy.cpu().numpy().squeeze() * 10
        epsilon_xx_full = epsilon_xx.cpu().numpy().squeeze()
        epsilon_yy_full = epsilon_yy.cpu().numpy().squeeze()
        epsilon_xy_full = epsilon_xy.cpu().numpy().squeeze()
        epsilon_1_full = epsilon_1.cpu().numpy().squeeze()
        epsilon_3_full = epsilon_3.cpu().numpy().squeeze()
        magnitude = np.sqrt(U_full**2 + V_full**2)

    # Calculate Safety Factor
    sf, sf_values, failure_ratios = phi_c_reduction(net, fem_data_file)
    print(f"Calculated Safety Factor: {sf:.4f}")
    
    df_out = pd.DataFrame({
        'X': X,
        'Y': Y,
        'ux': U_full,
        'uy': V_full,
        'sigma_xx': sigma_xx_full,
        'sigma_yy': sigma_yy_full,
        'sigma_xy': sigma_xy_full,
        'sigma_1': sigma_1_sorted,
        'sigma_3': sigma_3_sorted,
        'epsilon_xx': epsilon_xx_full,
        'epsilon_yy': epsilon_yy_full,
        'epsilon_xy': epsilon_xy_full,
        'epsilon_1': epsilon_1_full,
        'epsilon_3': epsilon_3_full,
        'magnitude': magnitude,
        'status': status_full,
        'strength_ratio': tau_rel_full,
        'tau_max': tau_max_full,
        'tau_mob': tau_mob_full,
        'safety_factor': sf,
    })

    df_out.to_csv(output_filename, index=False)
    print(f"Results saved to {output_filename}")



    
    plt.tight_layout()
    plt.savefig('PiNN_results.png')
    plt.show()
    return sf

if __name__ == "__main__":
    net = Net().to(device)
    
    optimizer = optim.Adam(net.parameters(), lr=0.001)
    
    fem_data_file = 'FEM4_data.csv'
    train(net, optimizer, n_epochs=1000, fem_data_file=fem_data_file)
    
    checkpoint = torch.load('best_model.pth', map_location=device)
    net.load_state_dict(checkpoint['net_state_dict'])
    
    try:
        sf = plot_results(net, fem_data_file=fem_data_file)
        print(f"\nSafety Factor: {sf:.4f}")
    except Exception as e:
        print(f"Error in plot_results: {str(e)}")

Using device: cuda


c:\Users\apisi\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "
